# Data Preprocessing

In [ ]:
!git clone https://github.com/indichealth/indic-health-demo.git

Cloning into 'indic-health-demo'...
remote: Enumerating objects: 231, done.
remote: Counting objects: 100% (121/121), done.
remote: Compressing objects: 100% (72/72), done.
remote: Total 231 (delta 55), reused 103 (delta 43), pack-reused 110
Receiving objects: 100% (231/231), 1.24 MiB | 18.96 MiB/s, done.
Resolving deltas: 100% (99/99), done.


In [ ]:
%cd indic-health-demo/Dataset

/content/indic-health-demo/Dataset


In [ ]:
import pandas as pd
import os

In [ ]:
data_path = 'IHQID-WebMD'

In [ ]:
train_df = pd.read_csv(os.path.join(data_path, 'train.csv'))
test_df = pd.read_csv(os.path.join(data_path, 'test.csv'))

In [ ]:
train_df = train_df[['question_hindi','Manual_Intent']]
test_df = test_df[['question_hindi','Manual_Intent']]

In [ ]:
num_classes = len(set(train_df['Manual_Intent']))

In [ ]:
train_df['question_hindi'][0]

'निस्टेटिन किस लिए निर्धारित किया गया है?'

In [ ]:
!pip install deep-translator

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 1.2 MB/s eta 0:00:00


In [ ]:
from deep_translator import GoogleTranslator

def translate_queries(data, src, tar):
  def translate(sentence, src, tar):
    return GoogleTranslator(source=src, target=tar).translate(sentence)

  s, t = src[:2], tar[:2]
  data[f'question_{tar}'] = data[f'question_{src}'].apply(lambda x: translate(x, s, t))
  return data

src = 'hindi'
tar = 'english'
train = translate_queries(train_df, src, tar).drop(columns=[f'question_{src}'])
test = translate_queries(test_df, src, tar).drop(columns=[f'question_{src}'])

In [ ]:
train

,Manual_Intent,question_english
0,drug,What is nystatin prescribed for?
1,other,Can douching after sex prevent me from getting...
2,drug,Does Percocet Cause Weight Gain
3,disease,Does 2 or 2 1/2 glasses of wine a day cause hi...
4,disease,Can too much buttermilk cause thrush?
...,...,...
715,other,Would the Affordable Care Act require an insur...
716,treatment plan,How can I use duct tape to get rid of warts?
717,treatment plan,What facial exercises can be done to help corr...
718,treatment plan,Is prenatal ultrasound safe?


# Model Creation

In [ ]:
!pip install transformers[torch]
!pip install datasets
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 61.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 58.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.2 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

In [ ]:
model_id = "emilyalsentzer/Bio_ClinicalBERT"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSequenceClassification.from_pretrained(model_id, num_labels=num_classes)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at emilyalsentzer/Bio_ClinicalBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from sklearn.preprocessing import LabelEncoder
from datasets import Dataset

def preprocess(data):
  lenc = LabelEncoder()
  queries = list(data['question_english'])
  labels = lenc.fit_transform(list(data['Manual_Intent']))

  dataset = Dataset.from_dict(
    {
        'text': queries,
        'label': labels
    }
  )

  max_length = 512
  def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=max_length)

  tokenized_dataset = dataset.map(tokenize_function, batched=True)
  return tokenized_dataset, labels

tokenized_train_dataset, train_labels = preprocess(train)
tokenized_test_dataset, test_labels = preprocess(test)

Map:   0%|          | 0/720 [00:00<?, ? examples/s]

Map:   0%|          | 0/241 [00:00<?, ? examples/s]

In [ ]:
tokenized_train_dataset

Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 720
})

# Model Training

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./output",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy="steps",
    eval_steps=500,
    num_train_epochs=10,
    load_best_model_at_end=True,
    learning_rate=5e-5,
    push_to_hub=False,
)

# Create a Trainer for training and evaluation
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,
)

trainer.train()

Step,Training Loss,Validation Loss
500,0.362500,1.471202


TrainOutput(global_step=900, training_loss=0.20579032156202529, metrics={'train_runtime': 662.9763, 'train_samples_per_second': 10.86, 'train_steps_per_second': 1.358, 'total_flos': 1894433616691200.0, 'train_loss': 0.20579032156202529, 'epoch': 10.0})

In [ ]:
# Evaluate the model on the test dataset
results = trainer.evaluate()

# Print the evaluation results
print(results)

{'eval_loss': 1.4712024927139282, 'eval_runtime': 7.7886, 'eval_samples_per_second': 30.943, 'eval_steps_per_second': 3.98, 'epoch': 10.0}


# Model Evaluation

In [ ]:
predictions = trainer.predict(tokenized_test_dataset)
predicted_labels = predictions.predictions.argmax(axis=1)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

print(classification_report(test_labels, predicted_labels))
print(confusion_matrix(test_labels, predicted_labels))

              precision    recall  f1-score   support

           0       0.80      0.79      0.79        76
           1       0.81      0.83      0.82        53
           2       0.76      0.73      0.74        73
           3       0.69      0.74      0.72        39

    accuracy                           0.77       241
   macro avg       0.77      0.77      0.77       241
weighted avg       0.77      0.77      0.77       241

[[60  2  6  8]
 [ 1 44  7  1]
 [11  5 53  4]
 [ 3  3  4 29]]
